<a href="https://colab.research.google.com/github/jacobgreen4477/Construction-Equipment-Oil-Condition-Classification-AI-Competition/blob/main/ETRI_v1_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> title : 제 4회 ETRI 휴먼이해 인공지능 논문경진대회 <br>
> author : hjy <br>

In our study, we used smartphones, smartwatches, sleep sensors, and self-recording apps to collect daily life logs and sleep health records of study participants in 2024.The data collection procedures and methods followed a similar approach to those used in previous studies. Here, we pu﻿blicly provide the following 12 data items, which comprise a total of 700 days' worth of lifelog data, strictly for non-commercial and academic research purposes only.
- mACStatus: Indicates whether the smartphone is currently being charged.
- mActivity: Value calculated by the Google Activity Recognition API.
- mAmbience: Ambient sound identification labels and their respective probabilities.
- mBle: Bluetooth devices around individual subject.
- mGps: Multiple GPS coordinates measured within a single minute using the smartphone.
- mLight: Ambient light measured by the smartphone.
- mScreenStatus: Indicates whether the smartphone screen is in use.
- mUsageStats: Indicates which apps were used on the smartphone and for how long.
- mWifi: Wifi devices around individual subject.
- wHr: Heart rate readings recorded by the smartwatch.
- wLight: Ambient light measured by the smartwatch.
- wPedo: Step data recorded by the smartwatch.

For the purpose of training a learning model to predict sleep health, fatigue, and stress, the following six metrics were derived from sleep sensor data and self-reported survey records. Each metric consists of values categorized into either two levels (0, 1) or three levels (0, 1, 2), depending on the specific metric. The detailed classification criteria for each metric's levels will be provided in a separate document.
- Q1: Overall sleep quality as perceived by a subject immediately after waking up.
- Q2: Physical fatigue of a subject just before sleep.
- Q3: Stress level experienced by a subject just before sleep.
- S1: Adherence to sleep guidelines for total sleep time (TST).
- S2: Adherence to sleep guidelines for sleep efficiency (SE).
- S3: Adherence to sleep guidelines for sleep onset latency (SOL, or SL).

In [1]:
! pip install haversine
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
from tqdm.auto import tqdm
from collections import Counter
from scipy.stats import entropy
from haversine import haversine  # 설치 필요: pip install haversine

warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# pandas 옵션
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%0.4f' % x)

In [4]:
def plot_rssi_by_subject_topN(df, TOPN):
    """
    subject_id별로 가장 자주 등장한 상위 N개 BSSID에 대해 RSSI 시계열 시각화
    - RSSI -70 이하 신호는 제외 (None 처리)
    - 선 투명도 50%
    """
    grouped = df.groupby("subject_id")

    for subject, group in grouped:
        timestamps = []
        bssid_counter = Counter()

        # 모든 BSSID 등장 횟수 카운트
        for _, row in group.iterrows():
            bssid_counter.update(row['bssid'])

        # TOP N BSSID 추출
        target_bssids = [b for b, _ in bssid_counter.most_common(TOPN)]
        time_series = {bssid: [] for bssid in target_bssids}

        # 시간별 RSSI 수집 (단, -60 이하는 제거)
        for _, row in group.iterrows():
            timestamps.append(pd.to_datetime(row['timestamp']))
            row_bssid = row['bssid']
            row_rssi = row['rssi']

            for bssid in target_bssids:
                if bssid in row_bssid:
                    idx = row_bssid.index(bssid)
                    rssi_value = row_rssi[idx]
                    if rssi_value > -60:
                        time_series[bssid].append(rssi_value)
                    else:
                        time_series[bssid].append(None)  # 약한 신호는 제외
                else:
                    time_series[bssid].append(None)  # 없는 BSSID

        # 시각화
        plt.figure(figsize=(16, 7))
        for bssid in target_bssids:
            plt.plot(timestamps, time_series[bssid], label=bssid, marker='o', alpha=0.5)

        plt.title(f"[{subject}] 시간별 RSSI 변화 (TOP {TOPN}, -60 이상만)", fontsize=14)
        plt.xlabel("시간")
        plt.ylabel("RSSI (dBm)")
        plt.xticks(rotation=45)
        plt.legend(title="BSSID", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

In [5]:
def filter_strong_rssi(df, threshold=-60):
    """
    각 row의 RSSI 값 중 threshold 이하인 항목 제거

    Parameters:
    - df: DataFrame with 'bssid' and 'rssi' as lists
    - threshold: int, RSSI 값 기준 (기본: -60)

    Returns:
    - 필터링된 DataFrame (in-place 수정 아님)
    """
    filtered_df = df.copy()

    def filter_row(row):
        bssids = row['bssid']
        rssis = row['rssi']
        # RSSI > threshold 조건 만족하는 항목만 추출
        filtered = [(b, r) for b, r in zip(bssids, rssis) if r > threshold]
        if filtered:
            new_bssids, new_rssis = zip(*filtered)
            return pd.Series({'bssid': list(new_bssids), 'rssi': list(new_rssis)})
        else:
            return pd.Series({'bssid': [], 'rssi': []})

    filtered_df[['bssid', 'rssi']] = filtered_df.apply(filter_row, axis=1)
    return filtered_df

In [6]:
def daily_wifi_features_by_user(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    features = []

    # subject_id + lifelog_date 기준 그룹화
    grouped = df.groupby(['subject_id', 'lifelog_date'])

    for (subject_id, date), group in grouped:
        scan_count = len(group)
        bssid_flat = sum(group['bssid'], [])  # flatten
        rssi_flat = sum(group['rssi'], [])    # flatten

        unique_bssid_count = len(set(bssid_flat))
        avg_rssi = sum(rssi_flat) / len(rssi_flat) if rssi_flat else None
        max_rssi = max(rssi_flat) if rssi_flat else None
        min_rssi = min(rssi_flat) if rssi_flat else None
        strong_rssi_ratio = sum(1 for r in rssi_flat if r > -60) / len(rssi_flat) if rssi_flat else 0
        empty_scan_count = sum(1 for b in group['bssid'] if len(b) == 0)

        # 가장 많이 탐지된 BSSID
        bssid_counter = Counter(bssid_flat)
        top_bssid, top_bssid_count = bssid_counter.most_common(1)[0] if bssid_counter else (None, 0)

        first_time = group['timestamp'].min()
        last_time = group['timestamp'].max()
        hour_span = (last_time - first_time).total_seconds() / 60  # 분 단위

        features.append({
            'subject_id': subject_id,
            'lifelog_date': date,
            'scan_count': scan_count,
            'unique_bssid_count': unique_bssid_count,
            'avg_rssi': avg_rssi,
            'max_rssi': max_rssi,
            'min_rssi': min_rssi,
            'strong_signal_ratio': strong_rssi_ratio,
            'empty_scan_count': empty_scan_count,
            'top_bssid': top_bssid,
            'top_bssid_count': top_bssid_count,
            #'time_first_seen': first_time,
            # 'time_last_seen': last_time,
            'hour_span_minutes': hour_span
        })

    return pd.DataFrame(features)

In [7]:
def generate_derived_features(df,
                             calculate_mean=True,
                             calculate_max=True,
                             calculate_entropy=True,
                             calculate_presence=False,
                             target_label=None):

    # 시간 필터링 함수
    def filter_night_hours(ts):
        hour = pd.to_datetime(ts).dt.hour
        return (hour >= 21) | (hour < 8)

    # 데이터 파싱 함수
    def parse_list(x):
        if isinstance(x, str):
            cleaned = x.strip('[]')
            items = [item.strip() for item in cleaned.split(',')]
            try:
                return list(map(float, items)) if '.' in cleaned else items
            except:
                return items
        return x

    # 1. 시간 필터링
    df = df[filter_night_hours(df['timestamp'])].copy()

    # 2. 데이터 전처리
    df = df.assign(
        labels=df['labels'].apply(parse_list),
        prob=df['prob'].apply(parse_list)
    ).explode(['labels', 'prob'])

    # 3. 타입 변환
    df['prob'] = pd.to_numeric(df['prob'], errors='coerce')
    df = df.dropna(subset=['prob'])

    # 4. 그룹화 기준
    group_keys = ['subject_id', 'lifelog_date']
    merged_df = pd.DataFrame()

    # 5. 평균 확률 계산
    if calculate_mean:
        mean_prob = df.groupby(group_keys + ['labels'])['prob'].mean().unstack().reset_index()
        merged_df = mean_prob.copy()

    # 6. 최대 확률 레이블 (조건 추가)
    if calculate_max:
        max_prob = df.loc[df.groupby(group_keys)['prob'].idxmax()][group_keys + ['labels', 'prob']]
        max_prob = max_prob.rename(columns={'labels': 'max_label', 'prob': 'max_prob'})

        # 조건 적용: max_prob < 0.1 → max_label = NaN
        max_prob['max_label'] = max_prob['max_label'].where(
            max_prob['max_prob'] >= 0.1,  # 조건
            np.nan  # 조건 미충족 시 NaN 할당
        )

        if not merged_df.empty:
            merged_df = pd.merge(merged_df, max_prob, on=group_keys, how='left')
        else:
            merged_df = max_prob.copy()

    # 7. 엔트로피 계산
    if calculate_entropy:
        entropy_df = df.groupby(group_keys + ['labels'])['prob'].mean() \
                     .groupby(group_keys).apply(entropy).reset_index(name='entropy')
        if not merged_df.empty:
            merged_df = pd.merge(merged_df, entropy_df, on=group_keys, how='left')
        else:
            merged_df = entropy_df.copy()

    # 8. 레이블 존재 여부
    if calculate_presence and target_label:
        presence_df = df.groupby(group_keys)['labels'] \
                      .apply(lambda x: (x == target_label).any().astype(int)) \
                      .reset_index(name=f'has_{target_label}')
        if not merged_df.empty:
            merged_df = pd.merge(merged_df, presence_df, on=group_keys, how='left')
        else:
            merged_df = presence_df.copy()

    # 9. 결과 정제
    if not merged_df.empty:
        merged_df = merged_df.fillna({f'has_{target_label}': 0})
        cols = [c for c in merged_df if c not in group_keys] + group_keys
        merged_df = merged_df[cols]

    return merged_df.reset_index(drop=True)

In [8]:
def calculate_daily_metrics(group):

    # 기본값 설정
    default_values = {
        'daily_avg_speed': 0,
        'daily_max_altitude': 0,
        'daily_lat_std': 0,
        'active_minutes': 0,
        'movement_ratio': 0,
        'centroid_lat': 0,
        'centroid_lon': 0,
        'altitude_change': 0,
        'total_distance(m)': 0,
        'max_speed': 0
    }

    try:


      """그룹별 모든 메트릭을 계산하는 통합 함수"""
      # 리스트 데이터 추출 (예외 처리 추가)
      all_speeds = np.concatenate(group['speed'].tolist()) if 'speed' in group else []
      all_alts = np.concatenate(group['altitude'].tolist()) if 'altitude' in group else []
      all_lats = np.concatenate(group['latitude'].tolist()) if 'latitude' in group else []
      all_lons = np.concatenate(group['longitude'].tolist()) if 'longitude' in group else []

      # 기본 통계
      avg_speed = np.mean(all_speeds) if len(all_speeds) > 0 else 0
      max_alt = np.max(all_alts) if len(all_alts) > 0 else 0
      lat_std = np.std(all_lats) if len(all_lats) > 0 else 0

      # 이동 특성
      active_mins = group.shape[0]  # 관측치 수 = 활동 분
      movement_ratio = (all_speeds > 1.0).mean() if len(all_speeds) > 0 else 0

      # 공간 특성
      centroid_lat = np.mean(all_lats) if len(all_lats) > 0 else 0
      centroid_lon = np.mean(all_lons) if len(all_lons) > 0 else 0
      alt_change = all_alts[-1] - all_alts[0] if len(all_alts) > 0 else 0

      # 이동 거리 계산 (Haversine)
      total_dist = 0.0
      if len(all_lats) > 1:
          for i in range(len(all_lats)-1):
              coord1 = (all_lats[i], all_lons[i])
              coord2 = (all_lats[i+1], all_lons[i+1])
              total_dist += haversine(coord1, coord2, unit='m')  # 미터 단위

    except Exception as e:
        print(f"Error processing group: {e}")
        return pd.Series(default_values)

    return pd.Series({
        # 기본 통계
        'daily_avg_speed': avg_speed,
        'daily_max_altitude': max_alt,
        'daily_lat_std': lat_std,

        # 이동 특성
        'active_minutes': active_mins,
        'movement_ratio': movement_ratio,

        # 공간 특성
        'centroid_lat': centroid_lat,
        'centroid_lon': centroid_lon,
        'altitude_change': alt_change,
        'total_distance(m)': total_dist,

        # 추가 메트릭
        'max_speed': np.max(all_speeds) if len(all_speeds) > 0 else 0
    })

### 데이터 읽기

In [9]:
path = '/content/drive/MyDrive/data/ch2025_data_items/'

# 1
ch2025_mACStatus = pd.read_parquet(path+'ch2025_mACStatus.parquet')
ch2025_mActivity = pd.read_parquet(path+'ch2025_mActivity.parquet')
ch2025_mAmbience = pd.read_parquet(path+'ch2025_mAmbience.parquet')
ch2025_mBle = pd.read_parquet(path+'ch2025_mBle.parquet')
ch2025_mGps = pd.read_parquet(path+'ch2025_mGps.parquet')
ch2025_mLight = pd.read_parquet(path+'ch2025_mLight.parquet')
ch2025_mScreenStatus = pd.read_parquet(path+'ch2025_mScreenStatus.parquet')
ch2025_mUsageStats = pd.read_parquet(path+'ch2025_mUsageStats.parquet')
ch2025_mWifi = pd.read_parquet(path+'ch2025_mWifi.parquet')
ch2025_wHr = pd.read_parquet(path+'ch2025_wHr.parquet')
ch2025_wLight = pd.read_parquet(path+'ch2025_wLight.parquet')
ch2025_wPedo = pd.read_parquet(path+'ch2025_wPedo.parquet')

# 2
train = pd.read_csv('/content/drive/MyDrive/data/ch2025_metrics_train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/ch2025_submission_sample.csv')

### ✅ mACStatus
- Indicates whether the smartphone is currently being charged.
- m_charging : 0/1 상태
- 핸드폰이 오랫 동안 충전했다는 의미?
 - 한 자리에 장시간 머물러 있었다.
 - 핸드폰을 장시간 사용하지 않았다.  

In [11]:
ch2025_mACStatus['lifelog_date'] = ch2025_mACStatus['timestamp'].astype(str).str[:10]
ch2025_mACStatus.head(1)

,subject_id,timestamp,m_charging,lifelog_date
0,id01,2024-06-26 12:03:00,0,2024-06-26


In [13]:
def process_mACStatus(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    df = df.sort_values(['subject_id', 'timestamp'])

    results = []

    for (subj, date), group in df.groupby(['subject_id', 'date']):
        status = group['m_charging'].values  # 0/1 상태
        times = group['timestamp'].values
        ratio_charging = status.mean()

        # 상태 전이 횟수
        transitions = (status[1:] != status[:-1]).sum()

        # 연속된 1 상태 길이들
        lengths = []
        current_len = 0
        for val in status:
            if val == 1:
                current_len += 1
            elif current_len > 0:
                lengths.append(current_len)
                current_len = 0
        if current_len > 0:
            lengths.append(current_len)

        avg_charging_duration = np.mean(lengths) if lengths else 0
        max_charging_duration = np.max(lengths) if lengths else 0

        results.append({
            'subject_id': subj,
            'lifelog_date': date,
            'charging_ratio': ratio_charging,
            'charging_transitions': transitions,
            'avg_charging_duration': avg_charging_duration,
            'max_charging_duration': max_charging_duration,
        })

    return pd.DataFrame(results)

ch2025_mACStatus2 = process_mACStatus(ch2025_mACStatus)
ch2025_mACStatus2.head()

,subject_id,lifelog_date,charging_ratio,charging_transitions,avg_charging_duration,max_charging_duration
0,id01,2024-06-26,0.2159,22,13.3636,41
1,id01,2024-06-27,0.1586,32,13.8750,65
2,id01,2024-06-28,0.1803,29,17.0667,76
3,id01,2024-06-29,0.2866,3,192.0000,328
4,id01,2024-06-30,0.1443,8,50.5000,91


### ✅ mActivity
- Value calculated by the Google Activity Recognition API.

In [14]:
ch2025_mActivity['lifelog_date'] = ch2025_mActivity['timestamp'].astype(str).str[:10]
ch2025_mActivity.head(1)

,subject_id,timestamp,m_activity,lifelog_date
0,id01,2024-06-26 12:03:00,4,2024-06-26


### mWifi
- Wifi devices around individual subject.

In [10]:
def extract_wifi_info(row):
    wifi_data = row['m_wifi']
    bssids = [item['bssid'] for item in wifi_data]
    rssis = [item['rssi'] for item in wifi_data]
    return pd.Series({'bssid': bssids, 'rssi': rssis})

ch2025_mWifi[['bssid', 'rssi']] = ch2025_mWifi.apply(extract_wifi_info, axis=1)
ch2025_mWifi['lifelog_date'] = ch2025_mWifi['timestamp'].astype(str).str[:10]
ch2025_mWifi = ch2025_mWifi.drop(columns=['m_wifi'])
ch2025_mWifi.head(1)

,subject_id,timestamp,bssid,rssi,lifelog_date
0,id01,2024-06-26 12:03:00,"[a0:0f:37:9a:5d:8b, a0:0f:37:9a:5d:8c, a0:0f:37:9a:5d:8d, a0:0f:37:9a:5d:8e, a0:0f:37:9a:5d:8f, a0:0f:37:96:56:ef, 88:36:6c:86:75:84, a0:0f:37:96:56:ee, a0:0f:37:96:56:ed, 86:25:19:b5:b2:a5, a0:0f:37:96:56:ec, 1e:39:29:8e:fb:e9, 52:c2:e8:c7:9b:e4, a0:0f:37:96:56:eb, 12:e3:c7:09:20:34, 58:86:94:4a:08:b8, 90:9f:33:28:d0:2e, 00:26:66:bc:4e:18, f6:0a:f4:43:4b:ba, 10:e3:c7:09:20:35, 10:e3:c7:09:20:34, 1c:39:29:48:04:92, 12:e3:c7:07:9d:df, 86:25:19:c3:44:07, a0:0f:37:9a:37:2f, a0:0f:37:9a:37:2e, a0:0f:37:9a:37:2d, 0a:09:b4:74:05:ec, a0:0f:37:9a:37:2c, a0:0f:37:9a:37:2b, 0a:09:b4:74:05:eb, c0:25:2f:d8:c1:a6, 16:7f:67:bb:fa:f8, 3c:f3:92:ff:00:01, 06:09:b4:74:05:ec, 06:09:b4:74:05:eb, 12:e3:c7:0a:74:d1, 88:36:6c:a9:6f:8e, 02:e3:c7:09:20:34, 00:09:b4:74:05:eb, 00:09:b4:74:05:ec, 00:1d:93:93:cf:fe, 8e:e2:ac:a5:9d:15]","[-78, -78, -78, -78, -78, -58, -72, -58, -58, -61, -58, -71, -82, -58, -88, -82, -78, -85, -45, -63, -89, -82, -83, -84, -76, -76, -76, -72, -76, -76, -59, -82, -79, -82, -72, -59, -78, -63, -88, -60, -72, -19, -72]",2024-06-26


In [ ]:
# wifi 약신호 제거
# ch2025_mWifi = filter_strong_rssi(ch2025_mWifi, threshold=-60)
# ch2025_mWifi.head(1)

In [ ]:
ch2025_mWifi_daily = daily_wifi_features_by_user(ch2025_mWifi)
ch2025_mWifi_daily.head()

,subject_id,lifelog_date,scan_count,unique_bssid_count,avg_rssi,max_rssi,min_rssi,strong_signal_ratio,empty_scan_count,top_bssid,top_bssid_count,hour_span_minutes
0,id01,2024-06-26,69,393,-70.1964,-19,-91,0.2309,0,86:25:19:9f:9b:be,19,716.0000
1,id01,2024-06-27,126,357,-69.0629,-26,-92,0.2701,0,04:09:a5:3a:c8:6a,54,1430.0000
2,id01,2024-06-28,118,376,-69.0941,-26,-92,0.2594,0,04:09:a5:3a:c8:6a,47,1430.0000
3,id01,2024-06-29,134,258,-67.7897,-24,-91,0.3063,0,04:09:a5:3a:c8:6a,117,1420.0000
4,id01,2024-06-30,108,242,-68.2999,-23,-90,0.2946,0,04:09:a5:3a:c8:6a,70,1310.0000


### mAmbience
- Ambient sound identification labels and their respective probabilities.

In [ ]:
# - mAmbience: Ambient sound identification labels and their respective probabilities.

def extract_labels_and_probs(row):
    items = row['m_ambience']
    labels = [item[0] for item in items]
    probs = [item[1] for item in items]
    return pd.Series({'labels': labels, 'prob': probs})

ch2025_mAmbience[['labels', 'prob']]  = ch2025_mAmbience.apply(extract_labels_and_probs, axis=1)
ch2025_mAmbience['lifelog_date'] = ch2025_mAmbience['timestamp'].astype(str).str[:10]
ch2025_mAmbience = ch2025_mAmbience.drop(columns=['m_ambience'])
ch2025_mAmbience.head(1)

,subject_id,timestamp,labels,prob,lifelog_date
0,id01,2024-06-26 13:00:10,"[Music, Vehicle, Motor vehicle (road), Outside, urban or manmade, Outside, rural or natural, Car, Speech, Inside, large room or hall, Truck, Sound effect]","[0.30902618, 0.081680894, 0.04035286, 0.037144363, 0.032663062, 0.03199804, 0.029806137, 0.01684492, 0.016206821, 0.01591479]",2024-06-26


In [ ]:
ch2025_mAmbience_daily = generate_derived_features(
    ch2025_mAmbience,
    calculate_mean=True,
    calculate_max=True,
    calculate_entropy=True,
    calculate_presence=True,
    target_label='Snoring'
)

In [ ]:
ch2025_mAmbience_daily.head()

,A capella,"Accelerating, revving, vroom",Accordion,Acoustic guitar,Afrobeat,Air brake,Air conditioning,"Air horn, truck horn",Aircraft,Aircraft engine,Alarm,Alarm clock,Ambient music,Ambulance (siren),Animal,Applause,Arrow,Artillery fire,Babbling,"Baby cry, infant cry",Baby laughter,Background music,Bagpipes,Bang,Banjo,Bark,Basketball bounce,Bass drum,Bass guitar,Bathtub (filling or washing),Beatboxing,"Bee, wasp, etc.","Beep, bleep",Bell,Bellow,Belly laugh,Bicycle,Bicycle bell,Bird,"Bird flight, flapping wings","Bird vocalization, bird call, bird song",Biting,Bleat,Blender,Bluegrass,Blues,"Boat, Water vehicle",Boiling,Boing,Boom,Bouncing,Bow-wow,Bowed string instrument,Brass instrument,Breaking,Breathing,"Burping, eructation","Burst, pop",Bus,Busy signal,Buzz,Buzzer,Cacophony,Camera,"Canidae, dogs, wolves",Cap gun,Car,Car alarm,Car passing by,Carnatic music,Cash register,Cat,Caterwaul,"Cattle, bovinae",Caw,Cello,Chainsaw,Change ringing (campanology),Chant,Chatter,Cheering,"Chewing, mastication","Chicken, rooster",Child singing,"Child speech, kid speaking",Children playing,Children shouting,Chime,"Chink, clink",Chirp tone,"Chirp, tweet",Choir,Chop,Chopping (food),Christian music,Christmas music,"Chuckle, chortle",Church bell,Civil defense siren,Clang,Clapping,Clarinet,Classical music,Clatter,Clickety-clack,Clicking,Clip-clop,Clock,Cluck,Coin (dropping),Computer keyboard,Conversation,Coo,Cough,Country,Cowbell,Crack,Crackle,Creak,Cricket,Croak,Crow,Crowd,"Crowing, cock-a-doodle-doo","Crumpling, crinkling",Crunch,Crushing,"Crying, sobbing",Cupboard open or close,"Cutlery, silverware",Cymbal,Dance music,"Dental drill, dentist's drill",Dial tone,Didgeridoo,Ding,Ding-dong,Disco,"Dishes, pots, and pans",Distortion,Dog,"Domestic animals, pets",Door,Doorbell,Double bass,Drawer open or close,Drill,Drip,Drum,Drum and bass,Drum kit,Drum machine,Drum roll,Dubstep,Duck,Echo,Effects unit,Electric guitar,Electric piano,"Electric shaver, electric razor",Electric toothbrush,Electronic dance music,Electronic music,Electronic organ,Electronic tuner,Electronica,Emergency vehicle,Engine,Engine knocking,Engine starting,Environmental noise,Eruption,Exciting music,Explosion,Fart,Field recording,Filing (rasp),Fill (with liquid),Finger snapping,Fire,Fire alarm,"Fire engine, fire truck (siren)",Firecracker,Fireworks,"Fixed-wing aircraft, airplane",Flamenco,Flap,Flute,"Fly, housefly",Foghorn,Folk music,Fowl,Frog,Frying (food),Funk,Fusillade,Gargling,Gasp,Gears,Giggle,Glass,Glockenspiel,Goat,Gobble,Gong,Goose,Gospel music,Groan,Growling,Grunt,Guitar,"Gunshot, gunfire",Gurgling,Gush,Hair dryer,Hammer,Hammond organ,Hands,Happy music,Harmonic,Harmonica,Harp,Harpsichord,Heart murmur,"Heart sounds, heartbeat",Heavy engine (low frequency),Heavy metal,Helicopter,Hi-hat,Hiccup,Hip hop music,Hiss,Honk,Hoot,Horse,House music,Howl,"Hubbub, speech noise, speech babble",Hum,Humming,"Ice cream truck, ice cream van",Idling,Independent music,Insect,"Inside, large room or hall","Inside, public space","Inside, small room",Jackhammer,Jazz,Jet engine,Jingle (music),Jingle bell,"Jingle, tinkle",Keyboard (musical),Keys jangling,Knock,Laughter,Lawn mower,Light engine (high frequency),Liquid,"Livestock, farm animals, working animals",Lullaby,Machine gun,Mains hum,Mallet percussion,Mandolin,Mantra,Maraca,"Marimba, xylophone",Mechanical fan,Mechanisms,Medium engine (mid frequency),Meow,Microwave oven,Middle Eastern music,Moo,Mosquito,Motor vehicle (road),"Motorboat, speedboat",Motorcycle,Mouse,Music,Music for children,Music of Africa,Music of Asia,Music of Bollywood,Music of Latin America,Musical instrument,"Narration, monologue","Neigh, whinny",New-age music,Noise,Ocean,Oink,Opera,Orchestra,Organ,"Outside, rural or natural","Outside, urban or manmade",Owl,Pant,Patter,Percussion,Piano,Pig,"Pigeon, dove",Ping,Pink noise,Pizzicato,Plop,Plucked string instrument,Police car (siren),Pop music,Pour,Power tool,"Power windows, electric windows",Printer,"Propeller, airscrew",Pulleys,Pulse,

### mGps
- Multiple GPS coordinates measured within a single minute using the smartphone.

In [ ]:
def extract_gps_info(row):
    m_data = row['m_gps']
    altitude = [item['altitude'] for item in m_data]
    latitude = [item['latitude'] for item in m_data]
    longitude = [item['longitude'] for item in m_data]
    speed = [item['speed'] for item in m_data]
    return pd.Series({'altitude': altitude, 'latitude': latitude, 'longitude': longitude, 'speed': speed})

ch2025_mGps[['altitude','latitude','longitude','speed']] = ch2025_mGps.apply(extract_gps_info, axis=1)
ch2025_mGps['lifelog_date'] = ch2025_mGps['timestamp'].astype(str).str[:10]
ch2025_mGps = ch2025_mGps.drop(columns=['m_gps'])
ch2025_mGps.head(1)

In [ ]:
ch2025_mGps_daily = ch2025_mGps.groupby(['subject_id', 'lifelog_date']).apply(calculate_daily_metrics).reset_index()
ch2025_mGps_daily.head()

,subject_id,lifelog_date,daily_avg_speed,daily_max_altitude,daily_lat_std,active_minutes,movement_ratio,centroid_lat,centroid_lon,altitude_change,total_distance(m),max_speed
0,id01,2024-06-26,0.6605,136.9000,0.0106,707.0000,0.1034,0.2361,0.1302,-6.7000,29113.6162,28.2200
1,id01,2024-06-27,1.0705,159.3000,0.0078,1439.0000,0.1401,0.2359,0.1130,0.0000,93470.9262,36.6356
2,id01,2024-06-28,0.8132,133.8000,0.0087,1418.0000,0.1105,0.2346,0.1108,0.0000,68632.0779,49.5476
3,id01,2024-06-29,0.5638,142.7000,0.0042,1440.0000,0.0865,0.2318,0.0959,0.6000,47104.6264,31.0703
4,id01,2024-06-30,0.4235,133.8000,0.0022,1440.0000,0.0494,0.2314,0.1042,-0.3000,41159.5137,33.8230


### mUsageStats
- mUsageStats: Indicates which apps were used on the smartphone and for how long.
- mScreenStatus: Indicates whether the smartphone screen is in use.

In [ ]:
def extract_mUsageStats_info(row):
    m_data = row['m_usage_stats']
    app_name = [item['app_name'] for item in m_data]
    total_time = [item['total_time'] for item in m_data]
    return pd.Series({'app_name': app_name, 'total_time': total_time})

ch2025_mUsageStats[['app_name', 'total_time']] = ch2025_mUsageStats.apply(extract_mUsageStats_info, axis=1)
ch2025_mUsageStats['lifelog_date'] = ch2025_mUsageStats['timestamp'].astype(str).str[:10]
ch2025_mUsageStats = ch2025_mUsageStats.drop(columns=['m_usage_stats'])
ch2025_mUsageStats.head(1)

,subject_id,timestamp,app_name,total_time,lifelog_date
0,id01,2024-06-26 13:00:00,"[ 캐시워크, NAVER, ✝️성경일독Q]","[69, 549, 7337]",2024-06-26


In [ ]:
ch2025_mUsageStats.loc[ch2025_mUsageStats['subject_id']=='id06',:]

,subject_id,timestamp,app_name,total_time,lifelog_date
25110,id06,2024-06-13 09:30:00,"[help-CNUH24, Withings, 설정, One UI 홈]","[71577, 138038, 14487, 232811]",2024-06-13
25111,id06,2024-06-13 17:40:00,"[AnyAUTH, One UI 홈]","[5318, 71832]",2024-06-13
25112,id06,2024-06-13 17:50:00,"[AnyAUTH, 메시지, YouTube, One UI 홈]","[5442, 1928, 105198, 1115]",2024-06-13
25113,id06,2024-06-13 18:00:00,[YouTube],[107102],2024-06-13
25114,id06,2024-06-13 18:20:00,"[카카오톡, YouTube, 시프티, One UI 홈]","[89047, 395785, 438459, 147282]",2024-06-13
...,...,...,...,...,...
28692,id06,2024-08-24 22:00:00,"[통화, 전화, 카카오톡, YouTube, One UI 홈]","[90763, 94292, 2775, 80858, 101879]",2024-08-24
28693,id06,2024-08-24 22:10:00,"[카카오톡, YouTube, One UI 홈]","[17602, 18527, 865]",2024-08-24
28694,id06,2024-08-24 22:30:00,"[YouTube, One UI 홈]","[533556, 1635]",2024-08-24
28695,id06,2024-08-24 22:40:00,"[넥슨플레이, YouTube, One UI 홈, 권한 관리자]","[14775, 65457, 87399, 204]",2024-08-24


In [ ]:
# 몇시까지 핸드폰 보다가 잠잤는지
# 통화, 전화 얼마나 했는지
# YouTube 얼마나 봤는지
# 메시지, 카카오톡 올마나 했는지
# NAVER 얼마나 했는지
# 평소보다 얼마나 많은 앱을 이용했는지
# 제외 -> 시스템 UI,One UI 홈

In [ ]:
ch2025_mUsageStats['app_name'].explode().value_counts().head(10)

,count
app_name,
One UI 홈,29114
카카오톡,17548
시스템 UI,16388
NAVER,7361
캐시워크,5042
✝️성경일독Q,4615
YouTube,4566
통화,4364
메시지,3845


In [ ]:
# 1. app_name 리스트 풀기 & 그룹별 빈도 계산
app_freq1 = (
    ch2025_mUsageStats.explode('app_name')  # 리스트 요소별로 행 분리
    .groupby(['subject_id', 'lifelog_date'])['app_name']  # 그룹 설정
    .value_counts()  # 앱별 사용 횟수 계산
    .reset_index(name='count')  # 결과 테이블화
)

# 2. 사용 시간 합계 추가 (선택 사항)
app_freq2 = app_freq1.merge(
    ch2025_mUsageStats.explode(['app_name', 'total_time'])  # 앱과 사용시간 동시 풀기
    .groupby(['subject_id', 'lifelog_date', 'app_name'])['total_time']
    .sum()
    .reset_index(name='total_time'),
    on=['subject_id', 'lifelog_date', 'app_name']
)

In [ ]:
app_freq1.head()

,subject_id,lifelog_date,app_name,count
0,id01,2024-06-26,✝️성경일독Q,40
1,id01,2024-06-26,One UI 홈,33
2,id01,2024-06-26,캐시워크,24
3,id01,2024-06-26,카카오톡,19
4,id01,2024-06-26,NAVER,11


In [ ]:
app_freq2.head()

,subject_id,lifelog_date,app_name,count,total_time
0,id01,2024-06-26,✝️성경일독Q,40,6964365
1,id01,2024-06-26,One UI 홈,33,5346126
2,id01,2024-06-26,캐시워크,24,1448492
3,id01,2024-06-26,카카오톡,19,2059689
4,id01,2024-06-26,NAVER,11,517215


In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

def generate_final_metrics(df):
    """모든 파생변수를 생성하는 통합 함수"""

    # 1. 기본 전처리
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['요일'] = df['lifelog_date'].dt.day_name()

    # 2. 리스트 평탄화 + 시스템 앱 필터링
    exploded_df = df.explode(['app_name', 'total_time'])
    exploded_df['total_time'] = exploded_df['total_time'].astype(int)
    filtered_df = exploded_df[~exploded_df['app_name'].isin(['시스템 UI', 'One UI 홈'])]

    # 3. 개인별 평균 기준 계산
    user_avg = filtered_df.groupby('subject_id')['app_name'].nunique().mean()

    # 4. 주요 파생변수 생성
    def calculate_daily_metrics(group):
        # 시간 관련
        last_use = group['timestamp'].max()
        date_str = group['lifelog_date'].iloc[0]

        # 앱별 사용시간 계산
        app_times = {
            '통화_시간': group[group['app_name'] == '통화']['total_time'].sum(),
            '전화_시간': group[group['app_name'] == '전화']['total_time'].sum(),
            'YouTube_시간': group[group['app_name'] == 'YouTube']['total_time'].sum(),
            '메신저_시간': group[group['app_name'].isin(['메시지','카카오톡'])]['total_time'].sum(),
            'NAVER_시간': group[group['app_name'] == 'NAVER']['total_time'].sum()
        }

        # 종합 통계
        return pd.Series({
            **app_times,
            '요일': group['요일'].iloc[0],
            '최종사용시간': last_use.strftime('%H:%M'),
            '추정취침시간': (last_use + timedelta(minutes=15)).strftime('%H:%M'),
            '고유앱수': group['app_name'].nunique(),
            '평균대비_앱사용량(%)': round((group['app_name'].nunique()/user_avg - 1)*100, 1),
            '총화면시간(분)': group['total_time'].sum()//60000
        })

    # 5. 시간대별 패턴 계산
    time_pattern = (
        filtered_df.set_index('timestamp')
        .groupby(['subject_id', 'lifelog_date', pd.Grouper(freq='3H')])
        .size()
        .unstack(fill_value=0)
        .add_prefix('시간대_')
        .reset_index()
    )

    # 6. 데이터 병합
    daily_stats = filtered_df.groupby(['subject_id', 'lifelog_date']).apply(calculate_daily_metrics).reset_index()
    final_df = pd.merge(daily_stats, time_pattern, on=['subject_id', 'lifelog_date'])

    # 7. 앱 사용 로그 병합 (선택적)
    app_logs = (
        filtered_df.groupby(['subject_id', 'lifelog_date', 'app_name'])
        ['total_time'].sum()
        .unstack(fill_value=0)
        .add_prefix('APP_')
    )

    return pd.merge(final_df, app_logs, on=['subject_id', 'lifelog_date'])

# 사용 예시
final_result = generate_final_metrics(ch2025_mUsageStats)
final_result.head()

,subject_id,lifelog_date,통화_시간,전화_시간,YouTube_시간,메신저_시간,NAVER_시간,요일,최종사용시간,추정취침시간,고유앱수,평균대비_앱사용량(%),총화면시간(분),시간대_2024-06-26 12:00:00,시간대_2024-06-26 15:00:00,시간대_2024-06-26 18:00:00,시간대_2024-06-26 21:00:00,시간대_2024-06-27 00:00:00,시간대_2024-06-27 03:00:00,시간대_2024-06-27 06:00:00,시간대_2024-06-27 09:00:00,시간대_2024-06-27 12:00:00,시간대_2024-06-27 15:00:00,시간대_2024-06-27 18:00:00,시간대_2024-06-27 21:00:00,시간대_2024-06-28 03:00:00,시간대_2024-06-28 06:00:00,시간대_2024-06-28 09:00:00,시간대_2024-06-28 12:00:00,시간대_2024-06-28 15:00:00,시간대_2024-06-28 18:00:00,시간대_2024-06-28 21:00:00,시간대_2024-06-29 03:00:00,시간대_2024-06-29 06:00:00,시간대_2024-06-29 09:00:00,시간대_2024-06-29 12:00:00,시간대_2024-06-29 15:00:00,시간대_2024-06-29 18:00:00,시간대_2024-06-29 21:00:00,시간대_2024-06-30 06:00:00,시간대_2024-06-30 09:00:00,시간대_2024-06-30 12:00:00,시간대_2024-06-30 15:00:00,시간대_2024-06-30 18:00:00,시간대_2024-06-30 21:00:00,시간대_2024-07-01 03:00:00,시간대_2024-07-01 06:00:00,시간대_2024-07-01 09:00:00,시간대_2024-07-01 12:00:00,시간대_2024-07-01 15:00:00,시간대_2024-07-01 18:00:00,시간대_2024-07-01 21:00:00,시간대_2024-07-02 03:00:00,시간대_2024-07-02 06:00:00,시간대_2024-07-02 09:00:00,시간대_2024-07-02 12:00:00,시간대_2024-07-02 15:00:00,시간대_2024-07-02 18:00:00,시간대_2024-07-02 21:00:00,시간대_2024-07-03 00:00:00,시간대_2024-07-03 03:00:00,시간대_2024-07-03 06:00:00,시간대_2024-07-03 09:00:00,시간대_2024-07-03 12:00:00,시간대_2024-07-03 15:00:00,시간대_2024-07-03 18:00:00,시간대_2024-07-03 21:00:00,시간대_2024-07-04 00:00:00,시간대_2024-07-04 03:00:00,시간대_2024-07-04 06:00:00,시간대_2024-07-04 09:00:00,시간대_2024-07-04 12:00:00,시간대_2024-07-04 15:00:00,시간대_2024-07-04 18:00:00,시간대_2024-07-04 21:00:00,시간대_2024-07-05 03:00:00,시간대_2024-07-05 06:00:00,시간대_2024-07-05 09:00:00,시간대_2024-07-05 12:00:00,시간대_2024-07-05 15:00:00,시간대_2024-07-05 18:00:00,시간대_2024-07-05 21:00:00,시간대_2024-07-06 06:00:00,시간대_2024-07-06 09:00:00,시간대_2024-07-06 12:00:00,시간대_2024-07-06 15:00:00,시간대_2024-07-06 18:00:00,시간대_2024-07-06 21:00:00,시간대_2024-07-07 06:00:00,시간대_2024-07-07 09:00:00,시간대_2024-07-07 12:00:00,시간대_2024-07-07 15:00:00,시간대_2024-07-07 18:00:00,시간대_2024-07-07 21:00:00,시간대_2024-07-08 00:00:00,시간대_2024-07-08 03:00:00,시간대_2024-07-08 06:00:00,시간대_2024-07-08 09:00:00,시간대_2024-07-08 12:00:00,시간대_2024-07-08 15:00:00,시간대_2024-07-08 18:00:00,시간대_2024-07-08 21:00:00,시간대_2024-07-09 03:00:00,시간대_2024-07-09 06:00:00,시간대_2024-07-09 09:00:00,시간대_2024-07-09 12:00:00,시간대_2024-07-09 15:00:00,시간대_2024-07-09 18:00:00,시간대_2024-07-09 21:00:00,시간대_2024-07-10 00:00:00,시간대_2024-07-10 03:00:00,시간대_2024-07-10 06:00:00,시간대_2024-07-10 09:00:00,시간대_2024-07-10 12:00:00,시간대_2024-07-10 15:00:00,시간대_2024-07-10 18:00:00,시간대_2024-07-10 21:00:00,시간대_2024-07-12 03:00:00,시간대_2024-07-12 06:00:00,시간대_2024-07-12 09:00:00,시간대_2024-07-12 12:00:00,시간대_2024-07-12 15:00:00,시간대_2024-07-12 18:00:00,시간대_2024-07-12 21:00:00,시간대_2024-07-13 06:00:00,시간대_2024-07-13 09:00:00,시간대_2024-07-13 12:00:00,시간대_2024-07-13 15:00:00,시간대_2024-07-13 18:00:00,시간대_2024-07-13 21:00:00,시간대_2024-07-14 06:00:00,시간대_2024-07-14 09:00:00,시간대_2024-07-14 12:00:00,시간대_2024-07-14 15:00:00,시간대_2024-07-14 18:00:00,시간대_2024-07-14 21:00:00,시간대_2024-07-15 00:00:00,시간대_2024-07-15 03:00:00,시간대_2024-07-15 06:00:00,시간대_2024-07-15 09:00:00,시간대_2024-07-15 12:00:00,시간대_2024-07-15 15:00:00,시간대_2024-07-15 18:00:00,시간대_2024-07-15 21:00:00,시간대_2024-07-16 00:00:00,시간대_2024-07-16 03:00:00,시간대_2024-07-16 06:00:00,시간대_2024-07-16 09:00:00,시간대_2024-07-16 12:00:00,시간대_2024-07-16 15:00:00,시간대_2024-07-16 18:00:00,시간대_2024-07-16 21:00:00,시간대_2024-07-17 06:00:00,시간대_2024-07-17 09:00:00,시간대_2024-07-17 12:00:00,시간대_2024-07-17 15:00:00,시간대_2024-07-17 18:00:00,시간대_2024-07-17 21:00:00,시간대_2024-07-19 00:00:00,시간대_2024-07-19 03:00:00,시간대_2024-07-19 06:00:00,시간대_2024-07-19 09:00:00,시간대_2024-07-19 12:00:00,시간대_2024-07-19 15:00:00,시간대_2024-07-19 18:00:00,시간대_2024-07-19 21:00:00,시간대_2024-07-21 00:00:00,시간대_2024-07-21 06:00:00,시간대_2024-07-21 09:00:00,시간대_2024-07-21 12:00:00,시간대_2024-07-21 15:00:00,시간대_2024-07-21 18:00:00,시간대_2024-07-21 21:00:00,시간대_2024-07-22 03:00:00,시간대_2024-07-22 06:00:00,시간대_

In [ ]:
def generate_final_metrics_v6(df):
    """날짜 포맷 오류 해결 버전"""

    # 1. 기본 전처리
    df = df.copy()
    df['lifelog_date'] = pd.to_datetime(df['lifelog_date'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['요일'] = df['lifelog_date'].dt.day_name()

    # 2. 리스트 평탄화 + 시스템 앱 필터링
    exploded_df = df.explode(['app_name', 'total_time'])
    exploded_df['total_time'] = exploded_df['total_time'].astype(int)
    filtered_df = exploded_df[~exploded_df['app_name'].isin(['시스템 UI', 'One UI 홈'])]

    # 3. 개인별 요일 평균 계산
    user_weekday_avg = (
        filtered_df.groupby(['subject_id', '요일'])
        .agg(avg_usage=('total_time', 'mean'))
        .reset_index()
    )

    # 4. 주요 파생변수 생성 (수정 부분)
    def calculate_daily_metrics(group):
        max_time = group['timestamp'].max()
        metrics = {
            '요일': group['요일'].iloc[0],
            '최종사용시간': max_time.strftime('%H:%M'),
            '추정취침시간': (max_time + pd.Timedelta(minutes=15)).strftime('%H:%M'),
            '총사용시간(분)': group['total_time'].sum() // 60000
        }

        # 타겟 앱별 사용시간
        target_apps = ['통화', '전화', 'YouTube', '메시지', '카카오톡', 'NAVER']
        for app in target_apps:
            metrics[f'{app}_사용시간'] = group[group['app_name'] == app]['total_time'].sum()

        # 요일별 평균 대비 사용량
        subject_id = group['subject_id'].iloc[0]
        current_weekday = metrics['요일']
        weekday_avg = user_weekday_avg[
            (user_weekday_avg['subject_id'] == subject_id) &
            (user_weekday_avg['요일'] == current_weekday)
        ]['avg_usage'].values

        metrics['요일평균대비'] = f"{round((group['total_time'].sum()/weekday_avg[0]-1)*100, 1)}%" if len(weekday_avg) >0 else 'N/A'

        return pd.Series(metrics)

    # 5. 요일별 패턴 계산
    def get_time_range(x):
        hour = x.hour
        if 0 <= hour < 6: return '새벽(0-6)'
        elif 6 <= hour < 12: return '오전(6-12)'
        elif 12 <= hour < 18: return '오후(12-18)'
        else: return '저녁(18-24)'

    filtered_df['시간대'] = filtered_df['timestamp'].apply(get_time_range)
    weekday_pattern = (
        filtered_df.groupby(['subject_id', 'lifelog_date', '요일', '시간대'])
        .size()
        .unstack(fill_value=0)
        .add_prefix('사용횟수_')
        .reset_index()
    )

    # 6. 데이터 병합
    daily_stats = filtered_df.groupby(['subject_id', 'lifelog_date']).apply(calculate_daily_metrics).reset_index()
    final_df = pd.merge(daily_stats, weekday_pattern, on=['subject_id', 'lifelog_date','요일'])

    return final_df

# 사용 예시
final_result = generate_final_metrics_v6(ch2025_mUsageStats)

In [ ]:
final_result.head()

,subject_id,lifelog_date,요일,최종사용시간,추정취침시간,총사용시간(분),통화_사용시간,전화_사용시간,YouTube_사용시간,메시지_사용시간,카카오톡_사용시간,NAVER_사용시간,요일평균대비,사용횟수_새벽(0-6),사용횟수_오전(6-12),사용횟수_오후(12-18),사용횟수_저녁(18-24)
0,id01,2024-06-26,Wednesday,23:30,23:45,334,552532,724428,6367,1432743,2059689,517215,17267.7%,0,0,94,78
1,id01,2024-06-27,Thursday,23:30,23:45,660,2211529,1006843,96552,1195425,5155199,2315289,33896.1%,20,128,109,104
2,id01,2024-06-28,Friday,23:40,23:55,561,1553451,1509327,1435106,1591361,4774502,2227733,30484.6%,17,106,118,102
3,id01,2024-06-29,Saturday,21:50,22:05,401,454607,7401,1707502,19179,981128,1755398,23777.7%,1,97,94,68
4,id01,2024-06-30,Sunday,22:50,23:05,627,2624571,1061335,952024,104117,5079988,1017024,34137.3%,0,85,137,120


### mBle (wifi랑 비슷하게 로직 적용)
- Bluetooth devices around individual subject.이탤릭체 텍스트

In [ ]:
def extract_mble_info(row):
    m_data = row['m_ble']
    address = [item['address'] for item in m_data]
    device_class = [item['device_class'] for item in m_data]
    rssi = [item['rssi'] for item in m_data]
    return pd.Series({'address': address, 'device_class': device_class, 'rssi': rssi})

ch2025_mBle[['address','device_class','rssi']] = ch2025_mBle.apply(extract_mble_info, axis=1)
ch2025_mBle['lifelog_date'] = ch2025_mBle['timestamp'].astype(str).str[:10]
ch2025_mBle = ch2025_mBle.drop(columns=['m_ble'])
ch2025_mBle.head(1)

,subject_id,timestamp,address,device_class,rssi,lifelog_date
0,id01,2024-06-26 12:13:00,"[00:15:7C:11:80:8D, 01:B1:D2:20:9E:3A, 04:33:1F:D9:C1:50, 06:5C:2D:BC:39:BE, 09:42:21:0D:AD:DF, 0B:66:0D:D5:9C:4A, 10:B5:88:E7:85:69, 13:F0:CA:3B:DB:EF, 1A:23:C0:8F:43:4D, 24:11:53:BB:62:89, 24:2D:F0:EE:1E:D0, 26:0C:48:28:15:77, 27:C1:C0:8B:82:C9, 28:9C:11:73:39:05, 34:40:DE:35:F8:65, 35:0A:59:BF:75:F5, 41:A6:C4:20:E3:2C, 42:6B:51:95:1B:D4, 44:B2:0B:78:04:0F, 45:37:48:E2:7F:CC, 4E:1B:C2:DF:C5:87, 4E:9F:1B:A9:56:5D, 50:63:B0:82:07:00, 53:13:6C:4F:04:D2, 54:15:89:95:27:44, 56:0E:2E:B0:D4:11, 5A:7A:2E:42:03:B1, 5A:9D:3E:AB:38:C6, 5E:A6:8E:B8:74:74, 5F:BC:08:0F:C1:6A, 62:E1:9D:41:F4:AE, 67:23:FE:88:69:A8, 68:EC:C5:0C:D1:C1, 6B:28:DA:C0:1B:29, 6F:0B:91:00:33:19, 70:7A:4B:82:44:90, 7B:62:D4:5B:59:D3, 7B:BE:A4:9D:FD:11, 7F:FD:C4:00:77:7D, C4:F0:92:C8:F1:8D, C7:3F:2C:7B:86:66]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1796, 0, 0, 0, 0, 0, 0, 7936, 0, 0, 0, 0, 0, 0, 0, 7936, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7936, 7936]","[-82, -61, -86, -75, -70, -89, -89, -77, -66, -37, -85, -63, -88, -30, -93, -72, -83, -77, -69, -87, -76, -66, -86, -69, -71, -61, -82, -83, -84, -87, -73, -88, -78, -75, -80, -88, -74, -72, -52, -87, -89]",2024-06-26


### 학습 & 테스트 데이터

In [ ]:
train2 = train.merge(ch2025_mWifi_daily,on=['subject_id','lifelog_date'],how='left')
train2.head()

,subject_id,sleep_date,lifelog_date,Q1,Q2,Q3,S1,S2,S3,scan_count,unique_bssid_count,avg_rssi,max_rssi,min_rssi,strong_signal_ratio,empty_scan_count,top_bssid,top_bssid_count,hour_span_minutes
0,id01,2024-06-27,2024-06-26,0,0,0,0,0,1,69.0000,393.0000,-70.1964,-19.0000,-91.0000,0.2309,0.0000,86:25:19:9f:9b:be,19.0000,716.0000
1,id01,2024-06-28,2024-06-27,0,0,0,0,1,1,126.0000,357.0000,-69.0629,-26.0000,-92.0000,0.2701,0.0000,04:09:a5:3a:c8:6a,54.0000,1430.0000
2,id01,2024-06-29,2024-06-28,1,0,0,1,1,1,118.0000,376.0000,-69.0941,-26.0000,-92.0000,0.2594,0.0000,04:09:a5:3a:c8:6a,47.0000,1430.0000
3,id01,2024-06-30,2024-06-29,1,0,1,2,0,0,134.0000,258.0000,-67.7897,-24.0000,-91.0000,0.3063,0.0000,04:09:a5:3a:c8:6a,117.0000,1420.0000
4,id01,2024-07-01,2024-06-30,0,1,1,1,1,1,108.0000,242.0000,-68.2999,-23.0000,-90.0000,0.2946,0.0000,04:09:a5:3a:c8:6a,70.0000,1310.0000


In [ ]:
test2 = test.merge(ch2025_mWifi_daily,on=['subject_id','lifelog_date'],how='left')
test2.head()

,subject_id,sleep_date,lifelog_date,Q1,Q2,Q3,S1,S2,S3,scan_count,unique_bssid_count,avg_rssi,max_rssi,min_rssi,strong_signal_ratio,empty_scan_count,top_bssid,top_bssid_count,hour_span_minutes
0,id01,2024-07-31,2024-07-30,0,0,0,0,0,0,115.0000,910.0000,-73.8566,-29.0000,-93.0000,0.1451,0.0000,86:25:19:9f:9b:be,42.0000,1400.0000
1,id01,2024-08-01,2024-07-31,0,0,0,0,0,0,135.0000,492.0000,-69.2624,-33.0000,-93.0000,0.2579,0.0000,04:09:a5:3a:c8:6a,48.0000,1430.0000
2,id01,2024-08-02,2024-08-01,0,0,0,0,0,0,124.0000,397.0000,-72.1787,-33.0000,-92.0000,0.2053,0.0000,86:25:19:9f:9b:be,49.0000,1428.0000
3,id01,2024-08-03,2024-08-02,0,0,0,0,0,0,132.0000,366.0000,-69.7404,-27.0000,-93.0000,0.2188,0.0000,04:09:a5:3a:c8:6a,53.0000,1430.0000
4,id01,2024-08-04,2024-08-03,0,0,0,0,0,0,107.0000,312.0000,-69.6403,-21.0000,-93.0000,0.2214,0.0000,04:09:a5:3a:c8:6a,77.0000,1420.0000
